In [20]:
import torch
!pip install d2l==1.0.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install matplotlib
%matplotlib inline
!pip install matplotlib-inline
import sys
!{sys.executable} -m pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import torch
import time
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
%matplotlib inline
from torchvision import transforms
import torchvision.datasets as datasets
from torchvision import datasets, transforms
import torch.optim as optim
from d2l import torch as d2l
from torch.optim import lr_scheduler
import torch.optim as optim

In [23]:
data_dir = './data' 
transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor(),transforms.Resize(64)])
CIFER10_trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(CIFER10_trainset, batch_size=10, shuffle=True)

Files already downloaded and verified


In [24]:
CIFER10_testset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(CIFER10_testset, batch_size=10, shuffle=False)

Files already downloaded and verified


In [25]:
class Residual(nn.Module): 
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1,
                                   stride=strides)
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [26]:
blk = Residual(3)
X = torch.randn(4, 3, 6, 6)
blk(X).shape

torch.Size([4, 3, 6, 6])

In [27]:
blk = Residual(6, use_1x1conv=True, strides=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

In [28]:
class ResNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [29]:
@d2l.add_to_class(ResNet)
def block(self, num_residuals, num_channels, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels))
    return nn.Sequential(*blk)

In [30]:
@d2l.add_to_class(ResNet)
def __init__(self, arch, lr=0.1, num_classes=10):
    super(ResNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1())
    for i, b in enumerate(arch):
        self.net.add_module(f'b{i+2}', self.block(*b, first_block=(i==0)))
    self.net.add_module('last', nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)

In [31]:
class ResNet18(ResNet):
    def __init__(self, lr=0.01, num_classes=10):
        super().__init__(((2, 64), (2, 128), (2, 256), (2, 512)),
                       lr, num_classes)

ResNet18()

ResNet18(
  (net): Sequential(
    (0): Sequential(
      (0): LazyConv2d(0, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (b2): Sequential(
      (0): Residual(
        (conv1): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Residual(
        (conv1): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): LazyBatchNorm2d(0, eps=1e-05, momentu

In [32]:
model = ResNet18()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [33]:
epoch_losses = [] 
model.train()
n_epochs=3
train_losses = np.zeros(n_epochs+1)
val_losses = np.zeros(n_epochs+1)
for epoch in range(n_epochs):
    running_loss = 0.0
    saved_loss = 0.0
    for i, data in enumerate(train_loader, 0):       
        inputs, labels = data
        inputs, labels = inputs, labels       
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        optimizer.step()
      
        running_loss += loss.item()
        train_losses[epoch+1]=running_loss/1000
        if i % 1000 == 999:  
            print('%d, %5d| loss: %.3f' %(epoch+1, i+1, running_loss/1000))
            saved_loss = running_loss
            running_loss = 0.0
    epoch_losses.append(saved_loss/5000)

1,  1000| loss: 1.757
1,  2000| loss: 1.469
1,  3000| loss: 1.301
1,  4000| loss: 1.177
1,  5000| loss: 1.081
2,  1000| loss: 0.981
2,  2000| loss: 0.937
2,  3000| loss: 0.912
2,  4000| loss: 0.850
2,  5000| loss: 0.817
3,  1000| loss: 0.744
3,  2000| loss: 0.736
3,  3000| loss: 0.706
3,  4000| loss: 0.690
3,  5000| loss: 0.686


In [34]:
train_loader = torch.utils.data.DataLoader(CIFER10_trainset, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(CIFER10_testset, batch_size=10, shuffle=False)

def validate(model, train_loader, test_loader):
    for name, loader in [("train", train_loader), ("val", test_loader)]:
        correct = 0
        total = 0

        with torch.no_grad(): 
            for imgs, labels in loader:
                imgs=imgs
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0]  
                correct += int((predicted == labels).sum()) 

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, test_loader)

Accuracy train: 0.79
Accuracy val: 0.75


In [35]:
pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
from torchinfo import summary

model = ResNet18()
batch_size = 10
summary(model, input_size=(batch_size, 3, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet18                                 [10, 10]                  --
├─Sequential: 1-1                        [10, 10]                  --
│    └─Sequential: 2-1                   [10, 64, 16, 16]          --
│    │    └─Conv2d: 3-1                  [10, 64, 32, 32]          9,472
│    │    └─BatchNorm2d: 3-2             [10, 64, 32, 32]          128
│    │    └─ReLU: 3-3                    [10, 64, 32, 32]          --
│    │    └─MaxPool2d: 3-4               [10, 64, 16, 16]          --
│    └─Sequential: 2-2                   [10, 64, 16, 16]          --
│    │    └─Residual: 3-5                [10, 64, 16, 16]          74,112
│    │    └─Residual: 3-6                [10, 64, 16, 16]          74,112
│    └─Sequential: 2-3                   [10, 128, 8, 8]           --
│    │    └─Residual: 3-7                [10, 128, 8, 8]           230,272
│    │    └─Residual: 3-8                [10, 128, 8, 8]           2